Pytorch implemenation

In [7]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load and preprocess the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=2
)

test_dataset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=False, num_workers=2
)

# Function to count the number of model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create the model instance
model = CNN().to(device)

# Print the model summary
print("Model parameters:", count_parameters(model))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training loop
def train():
    model.train()
    for epoch in range(10):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 199:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / 200))
                running_loss = 0.0

# Train the model
train()

# Evaluation on the test set
def evaluate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Print the accuracy
print("Validation accuracy:", evaluate(), "%")


100%|██████████| 9912422/9912422 [00:00<00:00, 72037480.06it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 65942130.55it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28060224.93it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4327698.49it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Model parameters: 27562
[1,   200] loss: 0.765
[1,   400] loss: 0.229
[1,   600] loss: 0.151
[1,   800] loss: 0.123
[2,   200] loss: 0.087
[2,   400] loss: 0.082
[2,   600] loss: 0.077
[2,   800] loss: 0.069
[3,   200] loss: 0.053
[3,   400] loss: 0.054
[3,   600] loss: 0.055
[3,   800] loss: 0.060
[4,   200] loss: 0.044
[4,   400] loss: 0.043
[4,   600] loss: 0.049
[4,   800] loss: 0.042
[5,   200] loss: 0.035
[5,   400] loss: 0.039
[5,   600] loss: 0.036
[5,   800] loss: 0.034
[6,   200] loss: 0.028
[6,   400] loss: 0.031
[6,   600] loss: 0.029
[6,   800] loss: 0.034
[7,   200] loss: 0.024
[7,   400] loss: 0.027
[7,   600] loss: 0.028
[7,   800] loss: 0.027
[8,   200] loss: 0.021
[8,   400] loss: 0.022
[8,   600] loss: 0.025
[8,   800] loss: 0.021
[9,   200] loss: 0.015
[9,   400] loss: 0.024
[9,   600] loss: 0.018
[9,   800] loss: 0.022
[10,   200] loss: 0.013
[10,   400] loss: 0.015
[10,   600] loss: 0.017
[

Tensorflow implementation

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(8, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation="relu"),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

# Print the model summary
model.summary()

# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

# Training loop
model.fit(train_images, train_labels, epochs=10, batch_size=64, verbose=1)

# Evaluation on the test set
_, accuracy = model.evaluate(test_images, test_labels)
print("Validation accuracy:", accuracy * 100, "%")


11490434/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        1168      
                                                                 
 flatten (Flatten)           (None, 1936)              0         
                                                                 
 dense (Dense)               (None, 64)                123968    
                                                                 
 dense_1 (Dense)             (None, 10)                65